In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data.head()


,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [2]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [21]:
# DISTRICT SUMMARY:

# Calculate total schools in the district
total_schools = len(school_data_complete["school_name"].unique())

# Calculate total number of students in the district
total_students = school_data_complete["student_name"].count()

# Calculate total budget in the district
total_budget = school_data["budget"].sum()

# Calculate average Math score in the district
average_math = student_data["math_score"].mean()

# Calculate average Reading Score in the district
average_reading = student_data["reading_score"].mean()

# Calculate the percentage of students that passed math in the district
passdata_math = (student_data.loc[student_data["math_score"] >=70,"math_score"]).count()
passpercent_math = (passdata_math/total_students)*100

# Calculate the percentage of students that passed reading in the district.
passdata_reading = (student_data.loc[student_data["reading_score"] >=70,"reading_score"]).count()
passpercent_reading = (passdata_reading/total_students)*100

# Calculate the percentage of students that passed math **and** reading in the district.
pass_total = (student_data.loc[(student_data["reading_score"] >=70) & (student_data["math_score"] >=70) ,
                               "reading_score"]).count()
passpercent_overall = (pass_total/total_students)*100

# Creating a summary dataframe for entire district.
district_summary_df = pd.DataFrame({"Total Schools": [total_schools],
                           "Total Students": [total_students],
                           "Total Budget": [total_budget],
                           "Average Math Score":[average_math],
                           "Average Reading Score": [average_reading],
                           "% Passing Math": [passpercent_math],
                            "% Passing Reading": [passpercent_reading],
                            "% Overall Passing": [passpercent_overall]})

# Format the financial columns for better visual
district_summary_df["Total Students"] = district_summary_df["Total Students"].astype(float).map(
    "{:,.0f}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].astype(float).map(
    "${:,.2f}".format)

# Print the district summary output 
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [23]:
# SCHOOL SUMMARY:

# Seggregate combined dataframe based on School name and generate columns: school type, no. of students,total budget,
# average math and reading scores by aggregae function 
school_names_df = school_data_complete.groupby(["school_name"]).agg({'type':pd.Series.mode,'student_name':['count'],
                                                                     'budget':pd.Series.mode,'math_score': ['mean'], 
                                                                   'reading_score': ['mean']})

# Rename these columns to desired text
school_names_df.columns = ['School Type', 'Total Students', 'Total School Budget', 'Average Math Score', 
                           'Average Reading Score' ]

# Calculate budget per student for every school and add it as a new column to the dataframe.
per_student_budget = school_names_df['Total School Budget']/school_names_df['Total Students']
school_names_df['Per Student Budget'] = per_student_budget

# Calculate passing percentage for Math for every school and add it as a new column to the dataframe.
pass_math = school_data_complete.groupby("school_name")['math_score'].apply(lambda x: (x>=70).sum())
school_names_df['% Passing Math'] = (pass_math/school_names_df['Total Students'])*100

# Calculate passing percentage for Reading for every school and add it as a new column to the dataframe.
pass_reading = school_data_complete.groupby("school_name")['reading_score'].apply(lambda x: (x>=70).sum())
school_names_df['% Passing Reading'] = (pass_reading/school_names_df['Total Students'])*100

# For Overall pass percentage based on both Math and Reading scores, first extract a dataframe from the merged dataframe
# consisting of data which matches both conditions.
math_reading = school_data_complete.loc[(school_data_complete["reading_score"] >=70) & 
                                        (school_data_complete["math_score"] >=70),:]

# Then groupby this new dataframe based on school name and preform a count function to get number of instances (=number of 
# students who passed both in Math and Reading).
math_reading_sorted = math_reading.groupby(['school_name']) 
math_reading_sorted_df = math_reading_sorted.count()

# Calculate overall passing percent and add a new column with this data to the initial dataframe
school_names_df['% Overall Passing'] = (math_reading_sorted_df['Student ID']/school_names_df['Total Students'])*100

# Rearrange columns and assign it to a new dataframe which will represent the school summary data. 
school_summary_df = school_names_df[['School Type','Total Students','Total School Budget','Per Student Budget',
                                  'Average Math Score','Average Reading Score','% Passing Math','% Passing Reading',
                                   '% Overall Passing']]
# Format the financial columns for better visual
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].astype(float).map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].astype(float).map("${:,.2f}".format)

# Print the school summary output
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [ ]:
# TOP PERFORMING SCHOOLS (By % Overall Passing)

